In [81]:
import spacy    
import unicodedata
import re

In [82]:
# Loads NLP English model
nlp = spacy.load('en')

In [83]:
processed = False
paper = "data/tm4/\"lacrosse virus\"/10.1186/1471-2334-11-29.txt" # oliver test paper
#paper = 'random_full.txt'

# Reads text
if 1:
    with open(paper, 'r') as file:
        text = file.read().replace('\n', ' ')
else:
    text = "In Fig. 2 and Tables 3 (available online only), 4 (available online only), 5, we reprint the estimated CFR for each Ebola outbreak (by virus) and for Marburg virus. The Ebola Zaire virus is the most lethal with an overall estimated CFR ranging from 69 to 88%2,5,25,38,43,49,50 (Table 3 (available online only)). The CFR of outbreaks due to Ebola Sudan virus ranged from 53 to 69%1,24,51–53 (Table 4 (available online only)), and the CFR of outbreaks due to Ebola Bundibugyo ranged from 34 to 42%19,46,47 (Table 5). For the ongoing outbreak in West Africa due to Ebola Zaire, the estimated CFR, as measured among confirmed and probable cases with definitive outcome (recovered or fatal), is approximately 70%, and varies little among the three most affected countries (Guinea, Liberia and Sierra Leone; Table 6 (available online only) and Data Citation 2)38. The CFR among EVD cases reported by Nigeria (n=20) was 40%54). A second, unrelated EVD outbreak occurred in Équateur province, DRC between July and October 2014 resulting in 69 confirmed and probable cases with a CFR of 74%49. The CFR for Marburg is approximately 80%55–57)."
    
text = unicodedata.normalize("NFKD", text) # replaces \xa0 with " "
text = re.sub(r'(?<=[.]) (?=[^\s])', r' ', text) # spaces out concatenated sentences
text

'The spatial epidemiology and clinical features of reported cases of La Crosse Virus infection in West Virginia from 2003 to 2007 http://dx.doi.org/10.1186/1471-2334-11-29  Abstract BackgroundLa Crosse virus (LACV) is a major cause of pediatric encephalitis in the United States. Since the mid-1980s, the number of reported cases of LACV infection in West Virginia has continued to rise and the state currently reports the most cases in the United States. The purpose of this study was to investigate and describe the spatial epidemiology and clinical presentation of LACV infection cases reported in West Virginia, as well as to provide a description of the environmental conditions present at the residences of the LACV infection cases.MethodsDescriptive and spatial analyses were performed on LACV infection cases reported to the West Virginia Department of Health from 2003 to 2007. Clinical and environmental variables were available for 96 cases and residence data were available for 68 of thes

In [84]:
def processText(text):
    global processed
    if not processed:
        processed = True
        text = text.replace('%','% ').replace('–',' – ').replace('-',' – ').replace('=',' = ').replace(',',', ')
        text = re.sub("[\[].*?[\]]", "", text)
        string1_protected = re.sub(r"(\d)\.(\d)", r"\1[PROTECTED_DOT]\2", text)  
        # now split (and remove empty lines)
        lines_protected = [line + "." for line in string1_protected.split(".") if line]   
        # now re-replace all "[PROTECTED_DOT]"s
        lines = [line.replace("[PROTECTED_DOT]", ".") for line in lines_protected]
        text = ' '.join(lines)
        text = re.sub(r'\s([?.!"](?:\s|$))', r'\1', text)
        text = re.sub(' +', ' ',text)
        return text
    else:
        print("Already processed")
        return text
text = processText(text)
text

'The spatial epidemiology and clinical features of reported cases of La Crosse Virus infection in West Virginia from 2003 to 2007 http://dx. doi. org/10.1186/1471 – 2334 – 11 – 29 Abstract BackgroundLa Crosse virus (LACV) is a major cause of pediatric encephalitis in the United States. Since the mid – 1980s, the number of reported cases of LACV infection in West Virginia has continued to rise and the state currently reports the most cases in the United States. The purpose of this study was to investigate and describe the spatial epidemiology and clinical presentation of LACV infection cases reported in West Virginia, as well as to provide a description of the environmental conditions present at the residences of the LACV infection cases. MethodsDescriptive and spatial analyses were performed on LACV infection cases reported to the West Virginia Department of Health from 2003 to 2007. Clinical and environmental variables were available for 96 cases and residence data were available for 

In [85]:
# Convert text into NLP object
textdoc = nlp(text)

In [86]:
# Named Entity Recognition
from spacy import displacy 
displacy.render(textdoc, style='ent')

In [87]:
sents = list(str(sent) for sent in list(textdoc.sents))

import pandas as pd
pd.set_option('display.max_colwidth', None)

# Put all sentences in the dataframe
sentsdf = pd.DataFrame(sents, columns = ['Sentence'])
sentsdf

,Sentence
0,The spatial epidemiology and clinical features of reported cases of La Crosse Virus infection in West Virginia from 2003 to 2007 http://dx.
1,doi.
2,org/10.1186/1471 – 2334 – 11 – 29 Abstract BackgroundLa Crosse virus (LACV) is a major cause of pediatric encephalitis in the United States.
3,"Since the mid – 1980s, the number of reported cases of LACV infection in West Virginia has continued to rise and the state currently reports the most cases in the United States."
4,"The purpose of this study was to investigate and describe the spatial epidemiology and clinical presentation of LACV infection cases reported in West Virginia, as well as to provide a description of the environmental conditions present at the residences of the LACV infection cases."
...,...
176,The majority of reported LACV infection cases during the study period had the presence of wooded areas and containers near their primary residence corroborating the results of a previous case – control study.
177,"Finally, the results of this study will be useful for guiding disease control strategies and will allow public health officials to target specific areas for interventions."
178,Authors’ original submitted files for images
179,Below are the links to the authors’ original submitted files for images.


In [88]:
# Listing keywords
numeric_keywords = ['distribution*','time','number*','ratio','proportion','period','±','total*','estimate*','%','review','parameter*','mean','period','value']
specific_keywords = ['cases','fatalities','n =','CFR','case-fatality','r0','reproduct*','infect*','infections','death*','transmis*','laten*','contact','infectious','incubat*','casualties','mortal*','morbid*','outbreak*','epideme*']
contextual_keywords = ['GPE','DATE','TIME','PRODUCT']
statistical_keywords = ['fatalities', 'deaths', 'cases']

# Creating a regular expression using keywords for searching and filtering 
numeric_regex = '|'.join(numeric_keywords)
specific_regex = '|'.join(specific_keywords)
statistical_regex = '|'.join(specific_keywords)

trait_keywords = numeric_keywords + specific_keywords
trait_regex = '|'.join(trait_keywords)
trait_regex

numReg = re.compile(numeric_regex)
specReg = re.compile(specific_regex)
traitReg = re.compile(trait_regex)
statReg = re.compile(statistical_regex)

In [89]:
import re

'''
Parses a sentence, looking for keywords.

args: sentence - string of sentence.
return: specific_keyword_count - total number of specific keywords in sentence
        numeric_keyword_count - total number of numeric keywords in sentence
        contextual_keyword_count - total number of contextual entity labels in sentence
        cardinality - total number of numeric entity labels in sentence
'''
def countKeywords(sentence):
    specific_keyword_count = numeric_keyword_count = contextual_keyword_count = cardinality = 0
    
    # counts specific and numeric keywords first
    for word in sentence.split():
        if re.match(specific_regex, word): specific_keyword_count += 1
        elif re.match(numeric_regex, word): numeric_keyword_count += 1
    
    # counts key entity labels
    sentence_obj = nlp(sentence)
    ent_labels = [ent.label_ for ent in sentence_obj.ents]
    for label in ent_labels:
        if label in contextual_keywords: contextual_keyword_count += 1
        elif label in ['CARDINAL']: cardinality += 1
        
    return specific_keyword_count, numeric_keyword_count, contextual_keyword_count, cardinality

In [90]:
sentsdf['SKC'],sentsdf['NKC'],sentsdf['CKC'],sentsdf['Cardinality'] = zip(*sentsdf['Sentence'].apply(countKeywords))
sentsdf

,Sentence,SKC,NKC,CKC,Cardinality
0,The spatial epidemiology and clinical features of reported cases of La Crosse Virus infection in West Virginia from 2003 to 2007 http://dx.,3,0,3,0
1,doi.,0,0,0,0
2,org/10.1186/1471 – 2334 – 11 – 29 Abstract BackgroundLa Crosse virus (LACV) is a major cause of pediatric encephalitis in the United States.,0,0,3,1
3,"Since the mid – 1980s, the number of reported cases of LACV infection in West Virginia has continued to rise and the state currently reports the most cases in the United States.",3,1,3,0
4,"The purpose of this study was to investigate and describe the spatial epidemiology and clinical presentation of LACV infection cases reported in West Virginia, as well as to provide a description of the environmental conditions present at the residences of the LACV infection cases.",5,0,1,0
...,...,...,...,...,...
176,The majority of reported LACV infection cases during the study period had the presence of wooded areas and containers near their primary residence corroborating the results of a previous case – control study.,2,1,0,0
177,"Finally, the results of this study will be useful for guiding disease control strategies and will allow public health officials to target specific areas for interventions.",0,0,0,0
178,Authors’ original submitted files for images,0,0,0,0
179,Below are the links to the authors’ original submitted files for images.,0,0,0,0


In [91]:
'''
Displays the text surrounding the sentence with the provided index. To refer to the context, perhaps.
args: idx - index of sentence in question.
    df = Dataframe, default is sentsdf.
    pm = plus or minus for the indices of surrounding sentences.
'''
def context(idx, pm = 1, df = sentsdf):
    n = len(df)
    
    if (idx-pm) < 0 and (idx+pm) > n:
        return df
    elif (idx-pm) < 0:
        return df.loc[0:idx+pm,:]
    elif (idx-pm) > n:
        return df.loc[idx-pm:n,:]
    return df.loc[idx-pm:idx+pm,:]

from collections import Counter

'''
Print entity labels and their occurences within a given sentence.
args: sentence - The sentence in question.
    from_keywords - Whether or not to only include info on labels in the contextual keywords. Does not by default.
return: list containing tuples of entity labels and their occurences.
'''
def printEntityLabels(sentence, from_keywords = False):
    sentence_obj = nlp(sentence)
    ent_labels = [ent.label_ for ent in sentence_obj.ents if (not from_keywords or ent.label_ in contextual_keywords)]
    labels = Counter(ent_labels).keys()
    counts = Counter(ent_labels).values()
    return list(zip(labels, counts))

'''
Prints all sentences from Dataframe with provided keywords.
args: filter_words: Words to filter results by. By default, the trait keywords.
    df - The Dataframe to filter. By default, sentsdf.
return: Returns Dataframe with only sentences including keywords from the list.
'''
def sentencesWith(filter_words, df=sentsdf):
    if isinstance(filter_words, str):
        filter_regex = filter_words
    elif isinstance(filter_words, list):
        filter_regex = '|'.join(filter_words)
    return df[df.Sentence.str.lower().str.contains(filter_regex)]

In [92]:
import numpy as np

f_out = open("testOutput.txt", "w")

dict_map = {
    # Infectious:
    "infectious":"infectious",
    "infection":"infectious",
    # Contact:
    "contact":"contact",
    # Incubation:
    "incubation":"incubation",
    # Latency:
    "latency":"latency",
    "latent":"latency",
    # Reproduction:
    "reproductive":"reproduction",
    "reproduction":"reproduction",
    "r0":"reproduction",
    # Cases/Deaths
    "fatalities":"deaths",
    "deaths":"deaths",
    "cases":"cases",
    # Transmission
    "transmission":"transmission",
    "transmi*":"transmission",
    # Case-fatality rate:
    "case - fatality":"CFR",
    "case-fatality":"CFR",
    "CFR":"CFR"
}

statsdf = pd.DataFrame([],columns=['Parameter','Estimates','n','Citation','Rule'])
print(statsdf)

from spacy.matcher import Matcher
bigMatcher = Matcher(nlp.vocab)
smallMatcher = Matcher(nlp.vocab)
smallMatcher.add("n_rule",None,[{'TEXT':'n'},{'TEXT':'='},{'LIKE_NUM':True}])
idx = None
error_indices = set()

def find_n(doc):
    n_match = smallMatcher(doc)
    if n_match:
        match_id, start, end = n_match[0]
        return int(doc[end-1].text)
    return np.NaN

# takes an NLP doc, a token of the matched estimate, a regex pattern produced by re.compile, and whether or not
# we search bidirectionally (backwards & forwards) or not (just backwards)
def proximitySearch(doc, est_token, regex, bidirectional = False):
    parameter = "None"
    # We start with the first token (index 0):
    closestIdx = abs(0 - est_token.i)
    for token in doc:
        # If the token comes before the estimate and matches a specific keyword:
        if (re.search(regex, token.text.lower())):
            if (((not bidirectional) & (token.i <= est_token.i)) or bidirectional):
                proximity_to_est = abs(token.i - est_token.i)
                # find the closest keyword to our estimate, as long as its within an arbitrary range (i said 15)
                if (proximity_to_est < closestIdx) & (proximity_to_est < 15):
                    closestIdx = token.i
                    parameter = token.text.lower()        
    return parameter

def proximitySearchPrevSentence(doc, regex, idx):
    parameter = "None"
    if (idx > 0):
        prevSent = sents_filt.loc[idx - 1].Sentence
        prevdoc = nlp(prevSent)
        closestIdx = 0
        for token in prevdoc:
            if (re.search(regex, token.text.lower())):
                    if (token.i > closestIdx):
                        closestIdx = token.i
                        parameter = token.text.lower()
    return parameter
    
def funnel_values(estimates, parameter, rule_name, doc):
    global statsdf
    
    n = find_n(doc) #could make this faster
    # Parameter IS in dictionary
    if parameter:
        # Checking if estimates already included, in which case doesn't add a new row
        if not ((statsdf['Parameter'] == parameter) &\
             (statsdf['Estimates'] == estimates)).any():
            new_row = {'Parameter':parameter,\
                       'Estimates':estimates,\
                       'n':n,\
                       'Citation':idx,\
                       'Rule':rule_name}
            statsdf = statsdf.append(new_row,ignore_index=True)
         
    # Parameter is NOT in dictionary
    elif not (idx in list(statsdf.Citation) and\
        estimates in list(statsdf.Estimates.loc[statsdf.Citation == idx])):
        statsdf.loc[len(statsdf)] = [parameter, estimates, n, idx, rule_name]
        
def pm_map(matcher, doc, id, matches):
    global error_indices
    
    for match_id, start, end in matches:
        try:
            string = str(doc[start:end])
            split_span = string.split()

            # Get the rule
            rule = nlp.vocab.strings[match_id]

            # Get our leftmost estimate token for search:
            avg_token = doc[end-3]

            # Compute interval
            moe = round(float(doc[end-1].text),2)
            avg = round(float(avg_token.text),2)
            estimates = (avg-moe,avg+moe)

            # Search for corresponding parameter
            parameter = proximitySearch(doc, avg_token, specReg)

            # Failed? Try searching last sentence:
            if (parameter == "None"):
                parameter = proximitySearchPrevSentence(doc, specReg, idx)
                rule = rule + " (prev sentence)"

            funnel_values(estimates, dict_map.get(parameter), rule, doc)
        except:
            error_indices.add(idx)
            print("Error at",idx,"with pm_map")

# days_map finds matches for "# days", then searches the rest of the sentences' tokens for specific keywords to
# store as the corresponding parameter.
def num_map(matcher, doc, id, matches):
    global error_indices
    
    for match_id, start, end in matches: 
        try:
            rule = nlp.vocab.strings[match_id]
            currSent = doc.text
            sentIdx = sents_filt.index[sents_filt.Sentence == currSent]
            est_token = doc[end-2]
            estimate = est_token.text

            # Search for corresponding parameter:
            parameter = proximitySearch(doc, est_token, specReg)

            # Failed? Try searching last sentence:
            if (parameter == "None"):
                parameter = proximitySearchPrevSentence(doc, specReg, idx)
                rule = rule + " (prev sentence)"

            funnel_values(estimate, dict_map.get(parameter), rule, doc)
        except:
            error_indices.add(idx)
            print("Error at",idx,"with num_map")
        
# and_map works like days map, but for ranges specified with "and"
def and_map(matcher, doc, id, matches):
    global error_indices
    
    for match_id, start, end in matches:
        try:
            rule = nlp.vocab.strings[match_id]
            # Find leftmost token (for searching):
            lower_est_token = doc[start]

            # Compute interval:
            lower = round(float(lower_est_token.text),2)
            upper = round(float(doc[end-1].text),2)
            estimates = (lower,upper)

            # Search for corresponding parameter:
            parameter = proximitySearch(doc, lower_est_token, specReg)

            # Failed? Try searching last sentence:
            if (parameter == "None"):
                parameter = proximitySearchPrevSentence(doc, specReg, idx)
                rule = rule + " (prev sentence)"

            funnel_values(estimates, dict_map.get(parameter), rule, doc)
        except:
            error_indices.add(idx)
            print("Error at",idx,"with and_map")
        
def cases_deaths_map(matcher, doc, id, matches):
    global error_indices
    
    for match_id, start, end in matches:
        try:
            estimates = int(float(doc[start].text.replace(",","")))
            parameter = doc[end-1].text.replace("fatalities","deaths")
            funnel_values(estimates, parameter, "cases_deaths_map", doc)
        except:
            error_indices.add(idx)
            print("Error at",idx,"with cases_deaths_map")
        
def r0_map(matcher, doc, id, matches):
    global error_indices
    
    for match_id, start, end in matches:
        try:
            funnel_values(round(float(doc[end-1].text),2),"reproduction","r0_map",doc)
        except:
            error_indices.add(idx)
            print("Error at",idx,"with r0_map")

Empty DataFrame
Columns: [Parameter, Estimates, n, Citation, Rule]
Index: []


In [93]:
bigMatcher.add("pm_rule", pm_map, [{"IS_ALPHA":True},{"IS_ALPHA":True},{"IS_ALPHA":True},\
           {"LIKE_NUM":True}, {"TEXT":"±"}, {"LIKE_NUM":True}])
bigMatcher.add("and_rule", and_map,\
               [{"IS_DIGIT":True}, {"TEXT":"and"}, {"IS_DIGIT":True}],\
               [{"IS_DIGIT":True}, {"TEXT":"–"}, {"IS_DIGIT":True}],\
               [{"IS_DIGIT":True}, {"TEXT":"to"}, {"IS_DIGIT":True}])
bigMatcher.add("num_rule", num_map,\
               [{"IS_ALPHA":True},{"IS_ALPHA":True},{"LIKE_NUM":True},{"TEXT":{"REGEX":"day*|%"}}],\
               [{"TEXT":{"REGEX":"was|is"}},{"LIKE_NUM":True},{"TEXT":{"REGEX":"day*|%"}}])
bigMatcher.add("cases_deaths_rule", cases_deaths_map, 
               [{"LIKE_NUM":True},{"TEXT":{"REGEX":"fatalities|deaths|cases"}}])
bigMatcher.add("r0_map",r0_map,[{"LOWER":"r0"},{"TEXT":"="},{"LIKE_NUM":True}])

In [94]:
statsdf = statsdf.iloc[0:0]

sents_filt = sentsdf
for idx in sents_filt.index:
    sentence = sents_filt.Sentence[idx]
    doc = nlp(sentence)
    matches = bigMatcher(doc)

None_indices =  [i for i, x in enumerate(list(statsdf.Parameter)) if x == None]
non_None_indices = list(set(list(statsdf.index))^set(None_indices))

statsdf_without_None = statsdf.iloc[non_None_indices]
statsdf_with_None = statsdf.iloc[None_indices]

statsdf = pd.concat([statsdf_without_None, statsdf_with_None])
statsdf

Error at 27 with and_map
Error at 31 with and_map
Error at 31 with and_map
Error at 31 with cases_deaths_map
Error at 84 with and_map
Error at 86 with and_map
Error at 88 with and_map


,Parameter,Estimates,n,Citation,Rule
0,infectious,"(2003.0, 2007.0)",NaN,0,and_rule
3,cases,96,NaN,6,cases_deaths_map
4,cases,189,NaN,11,cases_deaths_map
5,cases,505,NaN,11,cases_deaths_map
6,cases,"(1964.0, 1986.0)",NaN,27,and_rule (prev sentence)
7,cases,to,NaN,27,and_rule (prev sentence)
8,infectious,0.9,NaN,27,num_rule
10,2007,1987,NaN,31,cases_deaths_map
11,cases,555,NaN,31,cases_deaths_map
14,cases,31.4,NaN,31,num_rule (prev sentence)


In [95]:
mostSKCdf = sentsdf.sort_values(by="SKC", ascending=False).head(20)
matchesdf = sentsdf.iloc[list(set(statsdf.Citation).union(error_indices))].sort_index(inplace=False)
relevantdf_indices = list(set(mostSKCdf.index)-set(matchesdf.index))
relevantdf = pd.concat([matchesdf, sentsdf.iloc[relevantdf_indices].sort_index(inplace=False)])
relevantdf

,Sentence,SKC,NKC,CKC,Cardinality
0,The spatial epidemiology and clinical features of reported cases of La Crosse Virus infection in West Virginia from 2003 to 2007 http://dx.,3,0,3,0
2,org/10.1186/1471 – 2334 – 11 – 29 Abstract BackgroundLa Crosse virus (LACV) is a major cause of pediatric encephalitis in the United States.,0,0,3,1
6,Clinical and environmental variables were available for 96 cases and residence data were available for 68 of these cases.,2,0,0,2
11,"The county level cumulative incidence for those counties in the primary high – risk cluster ranged from 100.0 to 189.0 cases per 100, 000 persons (median 189.0) and the census tract level cumulative incidence for those counties in the high – risk cluster ranged from 61.7 to 505.9 cases per 100, 000 persons (median 99.0).",2,0,0,8
27,"From 1964 to 1986, there were 15 reported cases of LACV infection in West Virginia, comprising 0.9% of the nationally reported cases.",3,0,2,1
31,"Following this outbreak, from 1987 to 2007, West Virginia reported 555 cases of La Crosse encephalitis to the Centers for Disease Control and Prevention (CDC), accounting for 31.4% of the total cases reported in the United States (CDC, unpublished data).",3,1,4,1
53,"Census tracts are subdivisions of a county typically containing between 2, 500 and 8, 000 persons and are relatively homogeneous with respect to population characteristics, economic status and living conditions.",0,0,1,3
56,"Statistical and geographic analysesCumulative incidences were calculated at both the county and the census tract levels, and spatial analyses were performed on 68 cases 15 years and younger for which location data were available.",1,0,1,1
65,A maximum spatial cluster size of 10% of the population 15 years and younger was used.,0,0,1,0
84,"This cluster had a cumulative incidence ranging from 100 to 189 cases per 100, 000 persons (median 111) and a relative risk of 9.2 implying that the risk of LACV infection was 9.2 times higher in these three counties than in those counties in the rest of the state of West Virginia.",2,1,1,7
